In [1]:
%pip install -U langchain-mcp-adapters langgraph langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from dotenv import load_dotenv

load_dotenv("/home/aidan/work/meow-chat/.env")

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

server_params = StdioServerParameters(
    command="python3",
    args=["./math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "(3 + 5) x 12는 얼마지?"})

agent_response

  + Exception Group Traceback (most recent call last):
  |   File "/home/aidan/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_229480/4285305349.py", line 16, in <module>
  |     async with stdio_client(server_params) as (read, write):
  |   File "/home/aidan/miniconda3/envs/meow-chat/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/home/aidan/.local/lib/python3.10/site-packages/mcp/client/stdio/__init__.py", line 182, in stdio_client
  |     async with (
  |   File "/home/aidan/miniconda3/envs/meow-chat/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 781, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Exception Group Traceback (most re

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python3",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) 
agent = create_react_agent(model, await client.get_tools())
math_response = await agent.ainvoke({"messages": "(3 + 5) x 12는 얼마지?"})
weather_response = await agent.ainvoke({"messages": "뉴욕의 날씨는 어때?"})

In [ ]:
math_response

In [ ]:
weather_response